# Linq

Inspired by engineer Spock


In [ ]:
#r "nuget:AlgorithmLibrary.MicBai" 
using AlgorithmLibrary;

using System.IO;
using System.Text;
using System.Collections.Generic;
using System.Linq;


In [ ]:
public class ChessPlayer
{
    public static readonly string FileName = Path.Combine(Directory.GetCurrentDirectory(), "Data","Top100ChessPlayers.csv");

    public string FirstName { get; set; }
    public string LastName { get; set; }
    public int BirthYear { get; set; }
    public int Rating { get; set; }

    private string _country;
    public string Country
    {
        get
        {
            return _country;
        }
        set => _country = value;
    }
    public int Id { get; set; }
    
    public override string ToString()
    {
        return $"Full Name: {FirstName + " " + LastName}, Rating = {Rating}, from {Country}, Born in {BirthYear}";
    }

    public static ChessPlayer ParseFideCsv(string line)
    {
        string[] parts = line.Split(';');
        return new ChessPlayer()
        {
            Id = int.Parse(parts[0]),
            LastName = parts[1].Split(',')[0].Trim(),
            FirstName = parts[1].Split(',')[1].Trim(),
            Country = parts[3],
            Rating = int.Parse(parts[4]),
            BirthYear = int.Parse(parts[6])
        };
    }

    public static List<ChessPlayer> GetList()
    {
        List<ChessPlayer> players = File.ReadAllLines(FileName)
                .Skip(1)
                .Select(ParseFideCsv)
                .ToList();

        return players;
    }
}

In [ ]:
var file = ChessPlayer.FileName;

var list = File.ReadAllLines(file)
    .Skip(1)
    .Select(ChessPlayer.ParseFideCsv)
    .Where(player => player.BirthYear > 1988)
    .OrderByDescending(player => player.Rating)
    .ThenBy(p => p.Country)
    .Take(5);

list.Take(2).ForEach(player => Console.WriteLine(player));

### Select


In [ ]:

void SelectDemo()
{
    var list = ChessPlayer.GetList();
    
    // create a new list of names
    var names = list.Select(player => player.FirstName + " " + player.LastName).Take(5).ForEach(player => Console.WriteLine(player));

    // create a new list of anonymous objects
    var players = list.Select( player => new { 
        player.FirstName, 
        player.LastName, 
        player.Rating }).Take(5).ForEach(player => Console.WriteLine(player));

    // create a new list of anonymous objects with autogenerated index
    var players2 = list.Select( (player, index) => new { 
        Index = index,
        player.FirstName, 
        player.LastName, 
        player.Rating }).Take(5).ForEach(player => Console.WriteLine(player));

}

Demo1();

### SelectMany
SelectMany is befitial when regular Select returns a collection of collections.
SelectMany flattens the rsult of such a query joining the elements.

In [ ]:
public class Person
    {
        public string Name { get; set; }
        public List<string> PhoneNumbers { get; set; }
    }
    
public static void SelectManyDemo()
{
    IEnumerable<Person> people = new List<Person>()
    {
        new Person() {Name = "Bob", PhoneNumbers = new List<string>() {"123", "456", "789"}},
        new Person() {Name = "John", PhoneNumbers = new List<string>() {}},
        new Person() {Name = "Jeff", PhoneNumbers = new List<string>() {"879", "146"}},
        new Person() {Name = "Jon", PhoneNumbers = new List<string>() {"765", "481"}},
        new Person() {Name = "Buster", PhoneNumbers = new List<string>() {"294", "090"}},
    };

    IEnumerable<List<string>> phonesList = people.Select(p=>p.PhoneNumbers);
    foreach (var phone in phonesList)
    {
        Console.WriteLine($"Phone:{String.Join(",", phone)}");
    }

    IEnumerable<string> phoneNumbers = people.SelectMany(p=>p.PhoneNumbers);
    var personsWithPhoneNumbers = people.SelectMany(p=>p.PhoneNumbers,
        (person, phone)=>new {person.Name, Phone = phone});
    personsWithPhoneNumbers.ForEach(p => Console.WriteLine($"Person:{p.Name}, Phone:{p.Phone}"));
}

SelectManyDemo()


### Count

In [ ]:
void CountDemo()
{
    // if the we have a list we can use the Count property
    // if we have an IEnumerable we can use the Count() method
    var players = ChessPlayer.GetList();
    var count = players.Count; 
    Console.WriteLine($"Total number of players: {count}");

    var countWithPredicate = players.Count(player => player.BirthYear > 1988);
    Console.WriteLine($"Total number of players born after 1988: {countWithPredicate}");

    ChessPlayer at = players.ElementAt(10);
    Console.WriteLine($"Player at index 10: {at}");
}

CountDemo()

### Where

In [ ]:
void WhereDemo()
{
    var list = ChessPlayer.GetList();

    // create a new list of players from Russia using a lambda expression
    var filteredPlayers = list.Where(player => player.Country == "RUS")
        .Take(5)
        .ForEach(player => Console.WriteLine(player));

    Console.WriteLine("=====================================");

    // used index to filter the list for every 2nd
    var filteredPlayers2 = list.Where( (player, index) => index % 2 == 0)
        .Take(5)
        .ForEach(player => Console.WriteLine(player));

}

WhereDemo();

### TakeWhile, SkipWhile

In [ ]:
void TakeWhileDemo()
{
    // must be ToList() to use ForEach!!!!!!
    var list = RandomData.Integers(1, 100, 10).ToList(); 
    list.ForEach(number => Console.WriteLine(number));

    Console.WriteLine("=====================================");

    // create a new list of of numbers while the number is less than 5
    var whileList = list.TakeWhile(number => number < 50);
    Console.WriteLine(whileList.Count());
    whileList.ForEach(number => Console.WriteLine(number));

    Console.WriteLine("=====================================");

    // create a new list of of numbers while the number is less than 5
    var skipList = list.SkipWhile(number => number < 50);
    Console.WriteLine(skipList.Count());
    skipList.ForEach(number => Console.WriteLine(number));
}

TakeWhileDemo();

### First, Last, Single, FirstOrDefault, LastOrDefault
Attention:
* if not entry is found then First/Last dwill throw an exception, but FrstOrDefault/LastOrDefault will return null.
* Single will throw an exception if more than one element meets the condition, SngleOrDefault will retun null if no entry is found 

In [ ]:
void FindDemo()
{
    var list = ChessPlayer.GetList().OrderByDescending(p => p.Rating);
    //list.Take(5).ForEach(p=>Console.WriteLine(p));

    // create a new list of players from Russia using a lambda expression
    var firstPlayer = list.First(player => player.Country == "RUS");
    Console.WriteLine(firstPlayer);

    var lastPlayer = list.Last(player => player.Country == "RUS");
    Console.WriteLine(lastPlayer);
}

FindDemo();

### SequenceEqual


In [ ]:
public class ChessPlayerComparer : IEqualityComparer<ChessPlayer>
{
    public bool Equals(ChessPlayer x, ChessPlayer y)
    {
        return x.FirstName == y.FirstName &&
                x.LastName == y.LastName &&
                x.BirthYear == y.BirthYear &&
                x.Rating == y.Rating &&
                x.Country == y.Country &&
                x.Id == y.Id;
    }

    public int GetHashCode(ChessPlayer obj)
    {
        return obj.FirstName.GetHashCode() ^
                obj.LastName.GetHashCode() ^
                obj.BirthYear.GetHashCode() ^
                obj.Rating.GetHashCode() ^
                obj.Country.GetHashCode() ^
                obj.Id.GetHashCode();
    }
}

In [ ]:
void EquqlDemo()
{
    var list1 = ChessPlayer.GetList()
        .OrderByDescending(p => p.Rating)
        .Take(5)
        .ToList();

    var list2 = ChessPlayer.GetList()
        .OrderByDescending(p => p.Rating)
        .Take(5)
        .ToList();

    // compare list whether they are equal
    Console.WriteLine(list1.SequenceEqual(list2, new ChessPlayerComparer()));
}

EquqlDemo();



### Any

In [ ]:
void AnyDemo()
{
    bool any = ChessPlayer.GetList()
        .Any(player => player.Country == "RUS");
    Console.WriteLine("Any.." + any);

    //  result depends on the comparer
    bool contains = ChessPlayer.GetList()
        .Contains(new ChessPlayer()
        { FirstName = "Magnus", LastName = "Carlsen" }, new ChessPlayerComparer()); 
    Console.WriteLine("Contains.." + contains);

    bool all = ChessPlayer.GetList()
        .All(player => player.Rating > 1000); 
    Console.WriteLine("All.." + all);

}

AnyDemo();

### Distinct
One of a kind

In [ ]:
string str = "Hello World";

Console.WriteLine($"The string {str} contains the following characters");
str.ToCharArray().Distinct().ForEach(c => Console.WriteLine(c));

In [ ]:
public class DistinctCountryComparer : IEqualityComparer<ChessPlayer>
{
    public bool Equals(ChessPlayer x, ChessPlayer y)
        {
            return x.Country == y.Country;
        }

        public int GetHashCode(ChessPlayer obj)
        {
            return obj.Country.GetHashCode();
        }
}

var list = ChessPlayer.GetList();
var distinctCountries = list.Distinct(new DistinctCountryComparer());
distinctCountries.OrderBy(p=>p.Country).ForEach(player => Console.WriteLine(player.Country));


### Grouping

[microsoft - query key words](https://learn.microsoft.com/en-us/dotnet/csharp/language-reference/keywords/query-keywords)

In [ ]:
// query syntax
var list = from player in ChessPlayer.GetList()
    where player.BirthYear > 1988
    orderby player.Rating descending, player.Country
    select player;

list.Take(3).ForEach(player => Console.WriteLine(player));

In [ ]:
// select
var players = ChessPlayer.GetList();

var ratings = players.Select(player => player.Rating);
var lastNames = players.Select(player => player.LastName);
var fullNames = players.Select(player => player.LastName + " " + player.FirstName);

var anonymousType = players.Select((player, index) =>
{
    return new
    {
        Index = index,
        player.FirstName,
        player.LastName
    };
});

anonymousType.Take(3).ForEach(player => Console.WriteLine(player));


CSV to XML

In [ ]:
using System.Xml.Linq;

void CsvToXml()
{
    
    var file = ChessPlayer.FileName;

    var records = File.ReadAllLines(file)
                    .Skip(1)
                    .Select(ChessPlayer.ParseFideCsv)
                    .ToList();

    var doc = new XDocument();
    var players = new XElement("Players",
        records.Select(record=> new XElement("Player",
            new XAttribute("Id", record.Id),
            new XAttribute("Rating", record.Rating),
            new XAttribute("BirthYear", record.BirthYear),
            new XAttribute("Country", record.Country),
            new XAttribute("FirstName", record.FirstName),
            new XAttribute("LastName", record.LastName))
        ));
    doc.Add(players);

    file = Path.Combine(Directory.GetCurrentDirectory(), "Data","ChessPlayers.xml");
    doc.Save(file);
}

CsvToXml();

Read XML file

In [ ]:
using System.Xml.Linq;

List<ChessPlayer> XmlToList()
{
    var file = Path.Combine(Directory.GetCurrentDirectory(), "Data","ChessPlayers.xml");
    var doc = XDocument.Load(file);

    var players = doc.Descendants("Player") //IEnumerable
        .Select(player => new ChessPlayer()
        {
            Id = (int)player.Attribute("Id"),
            Rating = (int)player.Attribute("Rating"),
            BirthYear = (int)player.Attribute("BirthYear"),
            Country = (string)player.Attribute("Country"),
            FirstName = (string)player.Attribute("FirstName"),
            LastName = (string)player.Attribute("LastName")
        });

    //players.ForEach(player => Console.WriteLine(player));
    return players.ToList();
}

var list = XmlToList();
list.Take(5).ForEach(player => Console.WriteLine(player));

var file = Path.Combine(Directory.GetCurrentDirectory(), "Data","ChessPlayersFromXmlToCsv.csv");
list.WriteToCsv(file);


SQL
[Microsoft Entity Framework](https://learn.microsoft.com/en-us/ef/ef6/get-started)

In [ ]:
#r "nuget:EntityFramework"
#r "nuget:Microsoft.EntityFrameworkCore"
#r "nuget:Microsoft.EntityFrameworkCore.SqlServer"

using System;
using System.Data.Entity;

Database.SetInitializer(new DropCreateDatabaseAlways<ChessPlayerDb>());

class ChessPlayerDb : DbContext
{
    public DbSet<ChessPlayer> ChessPlayers { get; set; }
}


In [ ]:
void InsertData()
{
    var fileName = ChessPlayer.FileName;

    var records = File.ReadAllLines(fileName)
        .Skip(1)
        .Select(ChessPlayer.ParseFideCsv)
        .ToList();

    var db = new ChessPlayerDb();

    if (!db.ChessPlayers.Any())
    {
        db.ChessPlayers.AddRange(records);
    }

    db.SaveChanges();
}

InsertData()


In [ ]:
void QueryData()
{
    var db = new ChessPlayerDb();
    db.Database.Log = Console.WriteLine;

    var players = db.ChessPlayers
        .Where(player => player.BirthYear > 1988)
        .OrderByDescending(player => player.Rating)
        .ThenBy(p => p.Country)
        .Take(5)
        .ToList();

    players.ForEach(player => Console.WriteLine(player));
}

QueryData()